# Model Selection

**Authors**: Joel Velasco and Zachary Hall

In [21]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split

from rich import print
from rich.pretty import pprint
from torchmetrics import Accuracy


## Load the Data

In [22]:
GEM = pickle.load(open("GEM_final.pkl", "rb"))
PhysData = pickle.load(open("PhysData.pkl", "rb"))
GEM.fillna(0, inplace=True)
GEM

,MD11G1000200,MD11G1000100,MD11G1000500,MD11G1000600,MD11G1001100,MD11G1001000,MD11G1001200,MD11G1001300,MD11G1001600,MD11G1001400,...,MD17G1260200,MD17G1260700,MD17G1263500,MD17G1267700,MD17G1268500,MD17G1274900,MD17G1277100,MD17G1278000,MD17G1282800,MD17G1283300
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_001,6.312706,4.103259,30.300987,31.247893,0.000000,21.147564,7.575247,350.039525,8.837788,9.469058,...,0.000000,0.000000,0.946906,0.000000,0.946906,1.578176,0.315635,0.000000,0.0,0.000000
CC_1_19_002,8.109457,3.012084,45.876358,79.936079,9.731349,18.304204,5.792469,194.626975,2.780385,25.023468,...,0.231699,2.316988,2.780385,0.000000,1.390193,0.000000,0.695096,0.000000,0.0,2.316988
CC_1_19_003B,6.655276,7.320803,43.481135,70.767765,8.208173,16.194504,5.102378,207.644603,3.993165,17.303717,...,0.443685,1.109213,1.331055,0.665528,0.000000,0.887370,1.331055,0.000000,0.0,1.331055
CC_1_19_010,5.761445,4.172081,38.144739,55.627745,8.741503,13.509595,6.754798,236.219244,0.000000,16.092312,...,0.993353,0.198671,0.794682,0.198671,0.397341,0.000000,0.198671,0.000000,0.0,0.397341
CC_1_19_011,6.899737,6.232021,32.050393,58.313910,9.793176,20.254068,5.786877,249.058265,5.564304,21.366929,...,0.445144,0.445144,1.335433,0.445144,0.000000,0.000000,0.000000,0.000000,0.0,0.445144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RD_1_18_086C,8.066557,7.333233,34.710638,63.065806,8.310998,15.644231,9.777644,159.620045,8.555439,21.266377,...,0.244441,0.488882,0.000000,0.244441,0.733323,0.488882,0.244441,0.244441,0.0,0.488882
RD_1_18_087C,7.315559,10.594948,32.289365,56.758649,0.000000,18.162768,12.360772,133.445890,0.000000,12.865294,...,0.252261,0.504521,1.261303,0.000000,1.261303,0.000000,0.756782,0.000000,0.0,0.504521
RD_1_18_100D,5.680568,7.952795,42.604260,81.042771,8.142148,13.065306,6.248625,144.665133,7.574091,23.858386,...,0.189352,0.757409,2.082875,1.136114,0.757409,2.272227,0.757409,0.189352,0.0,1.704170


In [23]:
PhysData

,Variety,Week,Treatment,RNASample.Description,Weight_mean,Starch_mean,S.S._mean,mls_mean,TA_mean,Color-Green_mean,...,C2H4_median,C2H4_sd,has_high_c2h4_sample,has_missing_c2h4_sample,Harvest_date,Rep_group_id,Project.Code,Week.Offset,Week.Offset.Alt,Week.Offset.Mean
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_001,CC,1,Harvest,Week1_T0_Rep01,213.400000,1.000000,10.066667,11.047667,0.732460,NaN,...,0.110,0.252567,False,True,2019-08-27,CC_Harvest_1,CC-1-19,-5.0,-6.0,-5.0
CC_1_19_002,CC,1,Harvest,Week1_T0_Rep02,217.366667,1.000000,10.266667,11.717000,0.776837,NaN,...,0.110,0.252567,False,True,2019-08-27,CC_Harvest_1,CC-1-19,-5.0,-6.0,-5.0
CC_1_19_003B,CC,1,Harvest,Week1_T0_Rep03,243.550000,1.000000,10.066667,11.075000,0.734272,NaN,...,0.110,0.252567,False,False,2019-08-27,CC_Harvest_1,CC-1-19,-5.0,-6.0,-5.0
CC_1_19_010,CC,2,Harvest,Week2_T0_Rep01,247.733333,1.000000,10.633333,10.208333,0.676813,NaN,...,0.070,0.031400,False,True,2019-09-03,CC_Harvest_2,CC-1-19,-4.0,-5.0,-4.0
CC_1_19_011,CC,2,Harvest,Week2_T0_Rep02,237.150000,1.000000,10.533333,10.693333,0.708968,NaN,...,0.070,0.031400,False,True,2019-09-03,CC_Harvest_2,CC-1-19,-4.0,-5.0,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RD_1_18_086C,RD,8,Harvest,wk8 T0 Rep02,NaN,3.250000,13.046667,3.517733,0.247121,NaN,...,2.355,2.654717,True,True,2018-10-09,RD_Harvest_8,RD-1-18,4.0,3.0,3.0
RD_1_18_087C,RD,8,Harvest,wk8 T0 Rep03,NaN,2.866667,12.700000,3.259033,0.228947,NaN,...,2.355,2.654717,False,False,2018-10-09,RD_Harvest_8,RD-1-18,4.0,3.0,3.0
RD_1_18_100D,RD,9,Harvest,wk9 T0 Rep01,NaN,3.766667,13.433333,3.578900,0.251418,NaN,...,5.030,3.443148,True,True,2018-10-16,RD_Harvest_9,RD-1-18,5.0,4.0,4.0


## Train Test Split the Data

In [24]:
X = GEM
Y = PhysData.loc[:,['Starch_median']]
Y

,Starch_median
RNASample.ID,
CC_1_19_001,1.00
CC_1_19_002,1.00
CC_1_19_003B,1.00
CC_1_19_010,1.00
CC_1_19_011,1.00
...,...
RD_1_18_086C,3.00
RD_1_18_087C,3.00
RD_1_18_100D,3.50


In [25]:
# Get replicate groups.
group_ids = pd.Series(PhysData['Rep_group_id'].unique())

# Get train group.
train_grp = group_ids.sample(frac=0.8, replace=False, random_state = 42)
test_grp = group_ids[~group_ids.isin(train_grp)]

print(train_grp.shape)
print(test_grp.shape)

(34,)

(8,)

In [26]:
train_Y = PhysData[PhysData['Rep_group_id'].isin(train_grp)][['Starch_median']]
train_X = GEM.loc[train_Y.index]
train_X

,MD11G1000200,MD11G1000100,MD11G1000500,MD11G1000600,MD11G1001100,MD11G1001000,MD11G1001200,MD11G1001300,MD11G1001600,MD11G1001400,...,MD17G1260200,MD17G1260700,MD17G1263500,MD17G1267700,MD17G1268500,MD17G1274900,MD17G1277100,MD17G1278000,MD17G1282800,MD17G1283300
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_001,6.312706,4.103259,30.300987,31.247893,0.000000,21.147564,7.575247,350.039525,8.837788,9.469058,...,0.000000,0.000000,0.946906,0.000000,0.946906,1.578176,0.315635,0.000000,0.0,0.000000
CC_1_19_002,8.109457,3.012084,45.876358,79.936079,9.731349,18.304204,5.792469,194.626975,2.780385,25.023468,...,0.231699,2.316988,2.780385,0.000000,1.390193,0.000000,0.695096,0.000000,0.0,2.316988
CC_1_19_003B,6.655276,7.320803,43.481135,70.767765,8.208173,16.194504,5.102378,207.644603,3.993165,17.303717,...,0.443685,1.109213,1.331055,0.665528,0.000000,0.887370,1.331055,0.000000,0.0,1.331055
CC_1_19_010,5.761445,4.172081,38.144739,55.627745,8.741503,13.509595,6.754798,236.219244,0.000000,16.092312,...,0.993353,0.198671,0.794682,0.198671,0.397341,0.000000,0.198671,0.000000,0.0,0.397341
CC_1_19_011,6.899737,6.232021,32.050393,58.313910,9.793176,20.254068,5.786877,249.058265,5.564304,21.366929,...,0.445144,0.445144,1.335433,0.445144,0.000000,0.000000,0.000000,0.000000,0.0,0.445144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RD_1_18_086C,8.066557,7.333233,34.710638,63.065806,8.310998,15.644231,9.777644,159.620045,8.555439,21.266377,...,0.244441,0.488882,0.000000,0.244441,0.733323,0.488882,0.244441,0.244441,0.0,0.488882
RD_1_18_087C,7.315559,10.594948,32.289365,56.758649,0.000000,18.162768,12.360772,133.445890,0.000000,12.865294,...,0.252261,0.504521,1.261303,0.000000,1.261303,0.000000,0.756782,0.000000,0.0,0.504521
RD_1_18_100D,5.680568,7.952795,42.604260,81.042771,8.142148,13.065306,6.248625,144.665133,7.574091,23.858386,...,0.189352,0.757409,2.082875,1.136114,0.757409,2.272227,0.757409,0.189352,0.0,1.704170


In [27]:
train_Y

,Starch_median
RNASample.ID,
CC_1_19_001,1.00
CC_1_19_002,1.00
CC_1_19_003B,1.00
CC_1_19_010,1.00
CC_1_19_011,1.00
...,...
RD_1_18_086C,3.00
RD_1_18_087C,3.00
RD_1_18_100D,3.50


In [28]:
train = train_X.join(train_Y)
train.head()

,MD11G1000200,MD11G1000100,MD11G1000500,MD11G1000600,MD11G1001100,MD11G1001000,MD11G1001200,MD11G1001300,MD11G1001600,MD11G1001400,...,MD17G1260700,MD17G1263500,MD17G1267700,MD17G1268500,MD17G1274900,MD17G1277100,MD17G1278000,MD17G1282800,MD17G1283300,Starch_median
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_001,6.312706,4.103259,30.300987,31.247893,0.000000,21.147564,7.575247,350.039525,8.837788,9.469058,...,0.000000,0.946906,0.000000,0.946906,1.578176,0.315635,0.0,0.0,0.000000,1.0
CC_1_19_002,8.109457,3.012084,45.876358,79.936079,9.731349,18.304204,5.792469,194.626975,2.780385,25.023468,...,2.316988,2.780385,0.000000,1.390193,0.000000,0.695096,0.0,0.0,2.316988,1.0
CC_1_19_003B,6.655276,7.320803,43.481135,70.767765,8.208173,16.194504,5.102378,207.644603,3.993165,17.303717,...,1.109213,1.331055,0.665528,0.000000,0.887370,1.331055,0.0,0.0,1.331055,1.0
CC_1_19_010,5.761445,4.172081,38.144739,55.627745,8.741503,13.509595,6.754798,236.219244,0.000000,16.092312,...,0.198671,0.794682,0.198671,0.397341,0.000000,0.198671,0.0,0.0,0.397341,1.0
CC_1_19_011,6.899737,6.232021,32.050393,58.313910,9.793176,20.254068,5.786877,249.058265,5.564304,21.366929,...,0.445144,1.335433,0.445144,0.000000,0.000000,0.000000,0.0,0.0,0.445144,1.0


In [29]:
test_Y = PhysData[PhysData['Rep_group_id'].isin(test_grp)][['Starch_median']]
test_X = GEM.loc[test_Y.index]
test_X

,MD11G1000200,MD11G1000100,MD11G1000500,MD11G1000600,MD11G1001100,MD11G1001000,MD11G1001200,MD11G1001300,MD11G1001600,MD11G1001400,...,MD17G1260200,MD17G1260700,MD17G1263500,MD17G1267700,MD17G1268500,MD17G1274900,MD17G1277100,MD17G1278000,MD17G1282800,MD17G1283300
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_064,6.770406,8.404642,35.719727,68.871369,0.000000,18.910444,9.105028,135.408116,0.000000,16.108897,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.167311,0.233462,0.000000,0.233462
CC_1_19_065B,4.573083,9.415170,24.748448,82.046484,0.000000,18.830341,10.760195,123.473233,8.877161,21.789394,...,0.000000,0.269005,0.000000,0.000000,0.807015,0.000000,0.538010,0.000000,0.000000,0.807015
CC_1_19_066,0.000000,6.569503,36.325487,91.393380,6.955944,17.583082,8.888151,132.162942,11.206799,15.457654,...,0.193221,0.000000,0.579662,0.386441,0.966103,0.000000,0.579662,0.000000,0.000000,0.386441
CC_1_19_091,4.457944,6.050067,28.658214,41.076774,5.731643,22.926571,6.368492,147.430590,0.000000,7.323766,...,0.000000,0.000000,0.000000,0.955274,0.000000,1.273698,0.955274,0.955274,0.000000,0.318425
CC_1_19_092,6.785115,9.450696,34.894878,56.946503,0.000000,18.174416,12.358603,110.015797,0.000000,19.143718,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.696279,0.242326,0.242326,0.000000,0.242326
CC_1_19_093,4.915610,8.110757,27.281637,32.443028,0.000000,18.433539,7.373416,152.875481,22.366027,11.305904,...,0.000000,0.000000,0.000000,0.245781,1.228903,1.228903,2.457805,1.720464,0.000000,0.000000
GS_1_18_037C,7.775631,2.915861,36.934245,79.311431,15.356870,14.773698,5.442941,191.669293,4.859769,26.825925,...,2.332689,1.166345,0.000000,0.777563,1.555126,0.000000,1.166345,0.000000,4.470988,1.749517
GS_1_18_038C,8.382337,3.841904,42.435579,85.395054,13.795929,16.066145,6.985281,210.955472,4.016536,26.718698,...,1.397056,1.222424,0.698528,0.523896,1.746320,0.000000,1.047792,0.000000,4.889696,1.047792
GS_1_18_083D,9.624665,3.701794,38.005086,53.305835,13.326459,10.118237,4.688939,294.909597,6.910016,17.028253,...,0.000000,1.233931,0.987145,0.000000,0.000000,0.000000,0.000000,0.000000,2.961435,0.000000


In [30]:
test_Y

,Starch_median
RNASample.ID,
CC_1_19_064,4.50
CC_1_19_065B,4.50
CC_1_19_066,4.80
CC_1_19_091,5.50
CC_1_19_092,5.60
CC_1_19_093,5.90
GS_1_18_037C,1.00
GS_1_18_038C,1.00
GS_1_18_083D,1.40


In [31]:
test = test_X.join(test_Y)
test.head()

,MD11G1000200,MD11G1000100,MD11G1000500,MD11G1000600,MD11G1001100,MD11G1001000,MD11G1001200,MD11G1001300,MD11G1001600,MD11G1001400,...,MD17G1260700,MD17G1263500,MD17G1267700,MD17G1268500,MD17G1274900,MD17G1277100,MD17G1278000,MD17G1282800,MD17G1283300,Starch_median
RNASample.ID,,,,,,,,,,,,,,,,,,,,,
CC_1_19_064,6.770406,8.404642,35.719727,68.871369,0.000000,18.910444,9.105028,135.408116,0.000000,16.108897,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.167311,0.233462,0.0,0.233462,4.5
CC_1_19_065B,4.573083,9.415170,24.748448,82.046484,0.000000,18.830341,10.760195,123.473233,8.877161,21.789394,...,0.269005,0.000000,0.000000,0.807015,0.000000,0.538010,0.000000,0.0,0.807015,4.5
CC_1_19_066,0.000000,6.569503,36.325487,91.393380,6.955944,17.583082,8.888151,132.162942,11.206799,15.457654,...,0.000000,0.579662,0.386441,0.966103,0.000000,0.579662,0.000000,0.0,0.386441,4.8
CC_1_19_091,4.457944,6.050067,28.658214,41.076774,5.731643,22.926571,6.368492,147.430590,0.000000,7.323766,...,0.000000,0.000000,0.955274,0.000000,1.273698,0.955274,0.955274,0.0,0.318425,5.5
CC_1_19_092,6.785115,9.450696,34.894878,56.946503,0.000000,18.174416,12.358603,110.015797,0.000000,19.143718,...,0.000000,0.000000,0.000000,0.000000,1.696279,0.242326,0.242326,0.0,0.242326,5.6


## Convert Data to Tensors

train_XT = torch.tensor(train_X.values, dtype=torch.float32)
train_YT = torch.tensor(train_Y['Starch_median'].values, dtype=torch.float32)

test_XT = torch.tensor(test_X.values, dtype=torch.float32)
test_YT = torch.tensor(test_Y['Starch_median'].values, dtype=torch.float32)

print('x_train:', train_XT.shape)
print('y_train:', train_YT.shape)
print('x_valid:', test_XT.shape)
print('y_valid:', test_YT.shape)

## Define the Config

In [42]:
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig

data_config = DataConfig(
    target=['Starch_median'],
    continuous_cols=train.columns.tolist(),
)
trainer_config = TrainerConfig(
    batch_size=50,
    max_epochs=25,
    auto_lr_find=True,
    early_stopping=None,  # Monitor valid_loss for early stopping
    # early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    # early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    load_best=True,  # After training, load the best checkpoint
    progress_bar="none",  # Turning off Progress bar
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
    accelerator="cpu",
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="",
    dropout=0.1,
    initialization=(  # No additional layer in head, just a mapping layer to output_dim
        "kaiming"
    ),
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [46]:
%%time
from pytorch_tabular import model_sweep
import warnings
# Filtering out the warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="regression",
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        model_list="lite",
        common_model_args=dict(head="LinearHead", head_config=head_config),
        metrics = ["accuracy", "f1_score"],
        metrics_prob_input = [False, False],
        metrics_params = [{}, {"average": "macro"}],
        rank_metric=("accuracy", "higher_is_better"),
        progress_bar=True,
        verbose=False # If True, will print the results of each model
)

Output()

TypeError: accuracy() missing 1 required positional argument: 'task'